In [1]:
import requests, praw, re, time
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup 
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
stop_words = nltk.corpus.stopwords.words('english')
wpt = nltk.WordPunctTokenizer()
from config import r_client_id,r_secret_key,r_dev,r_username,r_pw


In [2]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
#print(reddit.user.me())

Version 7.2.0 of praw is outdated. Version 7.4.0 was released Friday July 30, 2021.


In [124]:
comment_lst = []
for submission in reddit.subreddit("politics+news").hot(limit=4): 
    if len(comment_lst) > 500000:
        break
    submission.comment_sort = 'best'
    for comment in submission.comments[:50]:   
        try:
            if 'I am a bot' not in comment.body: 
                comment_lst.append({'title':submission.title ,'body':comment.body, 'score':comment.score })
        except:
            pass
df = pd.DataFrame(comment_lst) 


In [125]:
df=df[['title','body']]
df.title.value_counts()

Texas Anti-Mask 'Freedom Rally' Organizer Fighting For His Life With COVID-19                       49
Biden vows to complete Afghanistan evacuation, hunt down ISIS leaders after Kabul attack            49
Lauren Boebert admits to campaign finance problems: “I under-reported a lot of stuff” to the FEC    49
Trump suggests Laden wasn’t ‘a monster’ and had ‘only one hit’ in new interview                     49
Name: title, dtype: int64

In [131]:
def normalize_document(doc): 
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc)
    doc = doc.lower().strip() 
    tokens = wpt.tokenize(doc)  
    doc = ' '.join([t for t in tokens if t not in stop_words])
    doc = ' '.join([t for t in tokens if len(t) < 25])
    return doc 
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(df.iloc[:]['body'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tv_matrix = tfidf.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()
idf = tfidf.idf_
vocab = tfidf.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

In [138]:
from sklearn.feature_extraction.text import CountVectorizer
bv = CountVectorizer(ngram_range=(2,2))
tv_matrix = bv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(tv_matrix, columns=vocab)

similarity_matrix = cosine_similarity(tv_matrix)
df_sim = pd.DataFrame(similarity_matrix)
km = KMeans(n_clusters=3)
km.fit_transform(df_sim) 
dff = pd.DataFrame(km.labels_, columns=['ClusterLabel'])
dff = pd.concat([df, dff], axis=1)
dff

,title,body,ClusterLabel
0,Lauren Boebert admits to campaign finance prob...,Under reporting financial stuff to the FEC is ...,1
1,Lauren Boebert admits to campaign finance prob...,That’s one issue. And you over-reported your ...,1
2,Lauren Boebert admits to campaign finance prob...,COLOR ME SURPRISED BECAUSE I NEVER SAW THIS CO...,1
3,Lauren Boebert admits to campaign finance prob...,Arrest her,1
4,Lauren Boebert admits to campaign finance prob...,">In a July 2020 letter to the FEC, Boebert's t...",1
...,...,...,...
191,"Biden vows to complete Afghanistan evacuation,...",Meanwhile Republicans predictably are doing ev...,1
192,"Biden vows to complete Afghanistan evacuation,...",It’s not a Trump vs Biden argument. They both...,1
193,"Biden vows to complete Afghanistan evacuation,...",Just went on Twitter and saw Donald Trump Jr's...,1
194,"Biden vows to complete Afghanistan evacuation,...","Man, there are some really immature MFers comm...",1


In [142]:
dff.loc[dff.ClusterLabel == 0].title.value_counts()
 

Trump suggests Laden wasn’t ‘a monster’ and had ‘only one hit’ in new interview    1
Name: title, dtype: int64

In [67]:

#print(norm_corpus)



,12 americans,12 us,15 injured,15 us,16 valor,17000 patients,17k hospitalized,18 billion,20000 people,2021 809,...,your hands,your own,your question,your serving,youre confused,youre part,youth challenge,zerg rush,zero regrets,zuck delete
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
similarity_matrix = cosine_similarity(bv_matrix)
df_sim = pd.DataFrame(similarity_matrix)
df_sim

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,1.000000,0.000000,0.000000,0.024056,0.000000,0.0,0.017568,0.0,0.02357,0.020211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028583,0.0,0.034021
1,0.000000,1.000000,0.000000,0.089087,0.023531,0.0,0.032530,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,0.000000,0.000000,1.000000,0.000000,0.020024,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,0.024056,0.089087,0.000000,1.000000,0.022011,0.0,0.060858,0.0,0.00000,0.035007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.049507,0.0,0.058926
4,0.000000,0.023531,0.020024,0.022011,1.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000
99,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000
100,0.028583,0.000000,0.000000,0.049507,0.000000,0.0,0.036155,0.0,0.00000,0.041595,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.070014
101,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.000000


In [53]:
from sklearn.cluster import KMeans

In [71]:
km = KMeans(n_clusters=3)
km.fit_transform(df_sim)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
cluster_labels

,ClusterLabel
0,1
1,3
2,2
3,1
4,0
...,...
98,2
99,2
100,1
101,2


In [72]:
dff = pd.concat([df, cluster_labels], axis=1)
dff

,title,body,ClusterLabel
0,We call upon Reddit to take action against the...,One new piece of information: it appears [the ...,1
1,Officer who shot Ashli Babbitt during Capitol ...,> It was the first time Byrd had ever shot his...,3
2,Officer who shot Ashli Babbitt during Capitol ...,She was trying to stay a zerg rush. People saw...,2
3,Officer who shot Ashli Babbitt during Capitol ...,> The Babbitt family’s attorney has described ...,1
4,Officer who shot Ashli Babbitt during Capitol ...,"He's right though, with one shot he stopped ev...",0
...,...,...,...
98,Facebook used facial recognition without conse...,That number seems … *low*,2
99,Facebook used facial recognition without conse...,"$5.5 million fine? Yep, that’ll teach ‘em!\n\n...",2
100,Facebook used facial recognition without conse...,That is like 2% of the civil liberties they ha...,1
101,Facebook used facial recognition without conse...,As if anyone on earth would be surprised by th...,2


In [ ]:
sns.scatterplot()

In [ ]:
dff=df 
ttitle=''
#dff=dff.sort_values('score', ascending=False )
for i, r in dff.iterrows():
    if ttitle != r.title:
        ttitle=r.title
        print('------------------------------------------------')
        print(f'{r.title} ')
        print('------------------------------------------------')
    print(f'{r.score}# {r.body} \n')

['An', 'gram', 'is', 'basically', 'a', '\ncollection', 'of', 'word', 'tokens', 'from', 'a', 'text', '\ndocument', 'such', 'that', 'these', 'tokens', 'are', '\ncontiguous', 'and', 'occur', 'in', 'a', 'sequence.']


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
for submission in reddit.subreddit("roastme"):
    #print(submission.title)
    comment_lst = []
    submission.comment_sort = 'best'
    for comment in submission.comments:
        try:
            #print(comment.body)
            d = {'Comment_Body':comment.body,
                'Author':comment.author,
                'Score':comment.score,
                'Time': time_convertion(comment.created_utc),
                'Symbol': symbol_pars(comment.body)
                }
            comment_lst.append(d)
        except:
            pass
#turns our list of dictionaries into a dataframe. Sort values by time posted
df = pd.DataFrame(comment_lst)
df.sort_values('Time', ascending=False)

In [ ]:
#FUNCTION to pars stock symbols out of comment body
def symbol_pars(s):
    match= re.findall('[A-Z\s]{3,4}', s)
    Match_split = ','.join(match)
    return Match_split

In [ ]:
#loop through subreddit Wallstreet Bets' Daily Discussion Thred, append into a list of dictionaries 


In [ ]:
#locate comments that contain BB 
test = df.loc[df['Comment_Body'].str.contains('BB')]
test.head()

In [ ]:
df.head()

In [ ]:
#locate comments that include AMC
AMCLoc = df.loc[df['Comment_Body'].str.contains('AMC') ]
AMCLoc.head()

In [ ]:
sia = SIA()
   
scores = sia.polarity_scores('I love cats ')
scores